<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-group-photo/blob/master/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# upload pictures
!wget -q https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/pictures/group_2/index.txt -O group_2.txt
!mkdir group_2


!xargs -i -a group_2.txt wget -q https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/pictures/group_2/{} -O group_2/{}
!ls -lah group_2

path = './group_2'

import os, sys

files = os.listdir(path)
files = list(map(lambda f: path + '/' + f, files))
print(files)

imgs = list(map(cv2.imread, files))

In [0]:
import numpy
import cv2
import dlib
from google.colab.patches import cv2_imshow

class Person:
  def __init__(self, id):
    self.id = id
    self.faces = []

	def faceVectorMatches(self, vector):
    return numpy.linalg.norm(self.id - vector) < 0.6

class Face:
 	def __init__(self, picture_data, picture_index, landmarks):
    self.picture_data = picture_data
    self.picture_index = picture_index
    self.landmarks = landmarks
    self.score_orientation = 0 # TODO: define metric for orientation
    self.score_eyes = None
    self.score_mouth = None
    self.score_all = None
  
  # sets and return score_all, parameters are for weighting the scores
  def scoreAll(self, w1 = 3, w2 = 2, w3 = 1):
    self.score_all = self.score_orientation * w1 + self.score_eyes * w2 + self.score_mouth * w3
    return self.score_all

# return the face given a picture and the coords of the face
def extractFace(pic, coords):
	return pic[coords.top():coords.bottom(), coords.left():coords:.right()]

# filenames
landmarks_filename = "shape_predictor_68_face_landmarks.dat"
facerecognition_filename = "dlib_face_recognition_resnet_model_v1.dat"

# TODO: fill pictures with filenames
# upload pictures 
pictures = []
cv_pictures = [cv2.imread(picture) for picture in pictures]
dlib_pictures = [cv2.cvtColor(cv_img,cv2.color_BGR2RGB) for cv_img in cv_pictures]

# init face detection and landmarks
face_detector = dlib.get_frontal_face_detector()
face_landmarks = dlib.shape_predictor(landmarks_filename)
face_recognition = dlib.face_recognition_model_v1(facerecognition_filename)

# list of every unique person
persons = []

# iterate each picture and search for faces
for pic_idx, picture in enumerate(dlib_pictures):
  # detect faces in the image
  faces = detector(picture, 0)
  
  # TODO: evlt Schleife angucken, was genau ist d?
  # d is the rectangle of the face
  # get landmarks of each face
  # get id for each face
  for k,d in enumerate(faces):
    landmarks = face_landmarks(picture,d)
    face_id = face_recognition.compute_face_descriptor(picture,landmarks)
    
    # check if person is already in persons list
    person = next(person for person in persons if person.faceVectorMatches(face_id), None)
    if person is None:
    	person = Person(face_id)
      persons.append(person)
      
    face = Face(extractFace(picture, d), pic_idx, landmarks)
    person.faces.append(face)
      

    

